## 10/03/2021
### Gashaw M. Goshu, Ph.D
####  Comparion of Avalon and Morgan Fingerprints using Random Forest and XGBoost Models for the prediction of HUMO-LUMO energy gap of organic molecules


In [1]:
import sklearn
from rdkit.Chem import AllChem
from rdkit import Chem
from rdkit.Chem import Descriptors
from sklearn.model_selection import train_test_split
from rdkit.Chem import rdMolDescriptors 
from rdkit.Chem import PandasTools
from rdkit.Avalon import pyAvalonTools
import pandas as pd
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score,mean_absolute_error, mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

import warnings
warnings.filterwarnings("ignore")

In [2]:
dataset = pd.read_csv('Orbital_Energies_input_data.csv')
dataset.tail(10)

,SMILES,Energygap
2894,C1CCC(=CC1)N1CCCC1,175.354880
2895,FC(F)(F)S(=O)(=O)OS(=O)(=O)C(F)(F)F,270.098071
2896,O=Cc1cccc2ccccc12,146.015069
2897,CN(C)c1ccc(cc1)C(=O)C,153.868344
2898,CC(C)(C)CC(C)(C)N,218.983071
2899,C(CP(c1ccccc1)c1ccccc1)P(c1ccccc1)c1ccccc1,168.649319
2900,Brc1cccc2sccc12,162.928319
2901,CCO[C@H]1C=Cc2ccccc2N1C(=O)OCC,165.098245
2902,s1ccc2ccccc12,167.958431
2903,Cc1cccc(C)c1O,188.369417


In [3]:
PandasTools.AddMoleculeColumnToFrame(dataset,'SMILES','Structure',includeFingerprints=True)

In [4]:
descriptor_names = list(rdMolDescriptors.Properties.GetAvailableProperties())
get_descriptors = rdMolDescriptors.Properties(descriptor_names)

In [5]:
# Let us see rdkit descriptors
print(len(descriptor_names))
descriptor_names

25


['exactmw',
 'lipinskiHBA',
 'lipinskiHBD',
 'NumRotatableBonds',
 'NumHBD',
 'NumHBA',
 'NumHeteroatoms',
 'NumAmideBonds',
 'FractionCSP3',
 'NumRings',
 'NumAromaticRings',
 'NumAliphaticRings',
 'NumSaturatedRings',
 'NumHeterocycles',
 'NumAromaticHeterocycles',
 'NumSaturatedHeterocycles',
 'NumAliphaticHeterocycles',
 'NumSpiroAtoms',
 'NumBridgeheadAtoms',
 'NumAtomStereoCenters',
 'NumUnspecifiedAtomStereoCenters',
 'labuteASA',
 'tpsa',
 'CrippenClogP',
 'CrippenMR']

In [6]:
# Calculate descriptors using smile strings
def smi_to_descriptors(smile):
    mol = Chem.MolFromSmiles(smile)
    descriptors = []
    if mol:
        descriptors = np.array(get_descriptors.ComputeProperties(mol))
    return descriptors
dataset['descriptors'] = dataset.SMILES.apply(smi_to_descriptors)

In [7]:
dataset[descriptor_names] = dataset['descriptors'].to_list()
dataset[descriptor_names]

,exactmw,lipinskiHBA,lipinskiHBD,NumRotatableBonds,NumHBD,NumHBA,NumHeteroatoms,NumAmideBonds,FractionCSP3,NumRings,...,NumSaturatedHeterocycles,NumAliphaticHeterocycles,NumSpiroAtoms,NumBridgeheadAtoms,NumAtomStereoCenters,NumUnspecifiedAtomStereoCenters,labuteASA,tpsa,CrippenClogP,CrippenMR
0,160.049985,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.250000,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,62.657894,0.00,3.01382,36.1810
1,122.013457,2.0,1.0,3.0,1.0,1.0,3.0,0.0,0.750000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,47.093099,37.30,1.09000,27.5898
2,361.108086,5.0,2.0,7.0,2.0,3.0,6.0,1.0,0.263158,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,151.127044,75.63,3.55450,96.2740
3,160.979905,1.0,2.0,0.0,1.0,1.0,3.0,0.0,0.000000,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,63.377826,26.02,2.57560,40.8744
4,156.151415,1.0,1.0,5.0,1.0,1.0,1.0,0.0,0.800000,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,70.128361,20.23,2.75130,49.5318
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2899,398.135324,0.0,0.0,7.0,0.0,0.0,2.0,0.0,0.076923,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,170.331941,0.00,5.25220,127.8780
2900,211.929533,0.0,0.0,0.0,0.0,1.0,2.0,0.0,0.000000,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,71.620979,0.00,3.66380,49.5250
2901,247.120843,4.0,0.0,3.0,0.0,3.0,4.0,1.0,0.357143,2.0,...,0.0,1.0,0.0,0.0,1.0,0.0,106.803952,38.77,3.03890,70.3910
2902,134.019021,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.000000,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,57.753431,0.00,2.90130,41.8250


In [8]:
# ML models
xgb = XGBRegressor(random_state=42)
rf = RandomForestRegressor(random_state=42)

In [10]:
# Data preprocessing using pipeline
# XGBoost
XGB = Pipeline(steps=[
        ('scale', StandardScaler()),
        ('model', xgb)])
# Data preprocessing using pipeline 
# Random Forest
RF = Pipeline(steps=[
        ('scale', StandardScaler()),
        ('model', rf)])

In [11]:
XGB_R2 =[]
XGB_rmse =[]

RF_R2 =[]
RF_rmse = []

for y in range(5):
    # Train-test split
    train, test = train_test_split(dataset)
    X_train = train[descriptor_names]
    y_train = train.Energygap
    X_test = test[descriptor_names]
    y_test = test.Energygap
    
    # Model training 
    XGB.fit(X_train,y_train)
    RF.fit(X_train,y_train)
    
    # Model testing
    xgb_preds = XGB.predict(X_test)
    rf_preds = RF.predict(X_test)
    # R^2 (coefficient of determination) regression score function: 
    r2_xgb = r2_score(y_test,xgb_preds)
    XGB_R2.append(r2_xgb)
    r2_rf = r2_score(y_test,rf_preds)
    RF_R2.append(r2_rf)
    
    # Model accuracy using root mean square error 
    rmse_XGB = np.sqrt(mean_squared_error(y_test, xgb_preds)) 
    XGB_rmse.append(rmse_XGB)
    
    rmse_RF = np.sqrt(mean_squared_error(y_test, rf_preds)) 
    RF_rmse.append(rmse_RF)
 
    
    print(f'XGBoost at split {y}: {r2_xgb:.2}')
    print(f'Random Forest at split {y}: {r2_rf:.2}')
    

XGBoost at split 0: 0.79
Random Forest at split 0: 0.78
XGBoost at split 1: 0.78
Random Forest at split 1: 0.78
XGBoost at split 2: 0.82
Random Forest at split 2: 0.81
XGBoost at split 3: 0.8
Random Forest at split 3: 0.82
XGBoost at split 4: 0.79
Random Forest at split 4: 0.8


In [12]:
# Average values
print(f'Average R2 for XGB : {np.mean(XGB_R2):.3} and rmse: {np.mean(XGB_rmse):.3}')
print(f'Average R2 for RF : {np.mean(RF_R2):.3} and rmse: {np.mean(RF_rmse):.3}')

Average R2 for XGB : 0.796 and rmse: 13.7
Average R2 for RF : 0.796 and rmse: 13.7


In [16]:
# Feature importance XGBoost
import eli5
from eli5.sklearn import PermutationImportance

perm = PermutationImportance(XGB, random_state=1).fit(X_test, y_test)
eli5.show_weights(perm, feature_names = X_test.columns.tolist())

Weight,Feature
0.6487 ± 0.0516,FractionCSP3
0.5700 ± 0.0601,CrippenMR
0.1355 ± 0.0174,tpsa
0.1333 ± 0.0121,NumAromaticRings
0.1251 ± 0.0182,labuteASA
0.0984 ± 0.0139,NumHBA
0.0962 ± 0.0146,exactmw
0.0427 ± 0.0114,CrippenClogP
0.0346 ± 0.0156,NumRotatableBonds
0.0171 ± 0.0048,NumHeteroatoms


In [17]:
# Feature importance XGBoost
import eli5
from eli5.sklearn import PermutationImportance

perm = PermutationImportance(RF, random_state=1).fit(X_test, y_test)
eli5.show_weights(perm, feature_names = X_test.columns.tolist())

Weight,Feature
0.6953 ± 0.0283,FractionCSP3
0.3729 ± 0.0343,NumAromaticRings
0.2366 ± 0.0220,CrippenMR
0.0628 ± 0.0037,tpsa
0.0317 ± 0.0065,NumHBA
0.0286 ± 0.0096,CrippenClogP
0.0285 ± 0.0040,exactmw
0.0239 ± 0.0048,NumRotatableBonds
0.0119 ± 0.0047,NumRings
0.0071 ± 0.0028,NumHeteroatoms


In [19]:
RF_mfpts = []
XGB_mfpts = []
RF_Avfpts = []
XGB_Avfpts = []

for x in range(0,5):
    # setup training and test sets
    train, test = train_test_split(dataset)
    train_morgan_fps = []
    train_avalon_fps=[]
    # for each random split, calculate Morgan & Avalon fingerprints for training set
    for i in train['SMILES']:
        mol = Chem.MolFromSmiles(i) 
        train_morgan_fps.append(AllChem.GetMorganFingerprintAsBitVect(mol,2,1024))
        train_avalon_fps.append(pyAvalonTools.GetAvalonFP(mol))
        
    test_morgan_fps = []
    test_avalon_fps = []
    # for each random split, calculate Morgan & Avalon fingerprints for test set
    for j in test['SMILES']:
        mol = Chem.MolFromSmiles(j) 
        test_morgan_fps.append(AllChem.GetMorganFingerprintAsBitVect(mol,2,1024))
        test_avalon_fps.append(pyAvalonTools.GetAvalonFP(mol))
       
    # For Morgan fingerprints
    train_X = np.array(train_morgan_fps)
    test_X = np.array(test_morgan_fps)
    # For Avalon fingerprints
    train2_X = np.array(train_avalon_fps)
    test2_X = np.array(test_avalon_fps)
    # Targets
    train_y = train.Energygap       
    test_y = test.Energygap
    # Define regressor models
    xgb = XGBRegressor(random_state=42)
    rf = RandomForestRegressor(random_state=42)
    # Train the model for Morgan fingerprints
    xgb.fit(train_X,train_y)
    rf.fit(train_X,train_y)
    
    pred_xgb = xgb.predict(test_X)
    pred_rf = rf.predict(test_X)
    r2_xgb = r2_score(test_y,pred_xgb)
    r2_rf = r2_score(test_y,pred_rf)
    
    # train the model for Avalon fingerprints
    xgb2 = XGBRegressor(random_state=42)
    rf2 = RandomForestRegressor(random_state=42)
    
    xgb2.fit(train2_X,train_y)
    rf2.fit(train2_X,train_y)
    
    pred_xgb2 = xgb2.predict(test2_X)
    pred_rf2 = rf2.predict(test2_X)
    r2_xgb2 = r2_score(test_y,pred_xgb2)
    r2_rf2 = r2_score(test_y,pred_rf2)
    
    RF_mfpts.append(r2_rf)
    RF_Avfpts.append(r2_rf2)
    XGB_mfpts.append(r2_xgb)
    XGB_Avfpts.append(r2_xgb2)
    
    print(f'XGBoost using Morgan FP at split {x}: {r2_xgb:.2}')
    print(f'Random Forest using Morgan FP at split {x}: {r2_rf:.2}')
    
    print("-----------------------------------------------")
    
    print(f'XGBoost using Avalon FP at  split {x}: {r2_xgb2:.2}')
    print(f'Random Forest using Avalon FP at split {x}: {r2_rf2:.2}')
    
    print("***********************************************")
   

XGBoost using Morgan FP at split 0: 0.8
Random Forest using Morgan FP at split 0: 0.83
-----------------------------------------------
XGBoost using Avalon FP at  split 0: 0.86
Random Forest using Avalon FP at split 0: 0.87
***********************************************
XGBoost using Morgan FP at split 1: 0.77
Random Forest using Morgan FP at split 1: 0.84
-----------------------------------------------
XGBoost using Avalon FP at  split 1: 0.85
Random Forest using Avalon FP at split 1: 0.84
***********************************************
XGBoost using Morgan FP at split 2: 0.82
Random Forest using Morgan FP at split 2: 0.83
-----------------------------------------------
XGBoost using Avalon FP at  split 2: 0.87
Random Forest using Avalon FP at split 2: 0.89
***********************************************
XGBoost using Morgan FP at split 3: 0.84
Random Forest using Morgan FP at split 3: 0.82
-----------------------------------------------
XGBoost using Avalon FP at  split 3: 0.88
Rand

In [27]:
# Average coefficient of determination
print(f'Morgan Fingerprints')
print(f'Random Forest & Morgan Fingerprints Average R^2 five splits: {np.mean(RF_mfpts):.2}')
print(f'XGBoost & Morgan Fingerprints Average R^2 five splits: {np.mean(XGB_mfpts):.2}')
print('~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~')
print(f'Avalon Fingerprints')
print(f'Random Forest & Avalon Fingerprints Average R^2 of five splits: {np.mean(RF_Avfpts):.2}')
print(f'XGBoost & Avalon Fingerprints Average R^2 of five splits: {np.mean(XGB_Avfpts):.2}')

Morgan Fingerprints
Random Forest & Morgan Fingerprints Average R^2 five splits: 0.83
XGBoost & Morgan Fingerprints Average R^2 five splits: 0.81
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Avalon Fingerprints
Random Forest & Avalon Fingerprints Average R^2 of five splits: 0.87
XGBoost & Avalon Fingerprints Average R^2 of five splits: 0.86
